# Python Fundamentals for Seismology — Notebook 8
## Reading Tables & Metadata with Pandas

*Python Fundamentals for Seismology*

Run cells in order. Edit parameters and re-run to explore.


## Learning objectives
- Understand working directories and file paths
- Read CSV tables with pandas
- Inspect DataFrames with head(), shape, dtypes
- Select and filter rows and columns
- Compute basic summary statistics
- Prepare tables for later seismology workflows (catalogs, station metadata)


## 0. Setup
We will use pandas, NumPy, and matplotlib.
These patterns are directly applicable to:
- event catalogs
- station metadata tables
- quality-control summaries


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option("display.max_rows", 10)
pd.set_option("display.max_columns", 10)


## 1. Working directories and file paths
Always know where your code is running and where files are being read from.


In [ ]:
# Current working directory
cwd = os.getcwd()
print("Current directory:", cwd)

# List files in the directory
print("\nFiles here:")
for name in os.listdir(cwd):
    print("  ", name)


### Changing directories (if needed)
In scripts, it is often better to use absolute or project-relative paths instead of changing directories.


## 2. Creating a small synthetic station metadata table
We will create a simple CSV-like table representing station metadata.
Later, you will replace this with real files (e.g., station lists, catalogs).


In [ ]:
data = {
    "station": ["MBGH", "MBBY", "MBLG", "MBFR", "MBRY"],
    "latitude": [16.72, 16.71, 16.69, 16.75, 16.73],
    "longitude": [-62.18, -62.20, -62.22, -62.15, -62.19],
    "elevation_m": [330, 250, 180, 410, 290],
    "start_year": [1996, 1997, 1998, 2000, 1999]
}

df = pd.DataFrame(data)
df


### Saving to CSV (for practice)


In [ ]:
csv_path = "station_metadata_example.csv"
df.to_csv(csv_path, index=False)
print("Wrote:", csv_path)


## 3. Reading a CSV file with pandas
The most common entry point for tabular data is `pd.read_csv`.


In [ ]:
df = pd.read_csv("station_metadata_example.csv")
df


### Inspecting a DataFrame


In [ ]:
print("shape:", df.shape)
print("\ncolumns:")
print(df.columns)
print("\ndtypes:")
print(df.dtypes)
print("\nfirst rows:")
df.head()


## 4. Selecting columns and rows
Two main access patterns:
- Column selection: `df['col']`
- Row/column selection: `.loc[]` and `.iloc[]`


In [ ]:
# Single column
stations = df["station"]
print(stations)

# Multiple columns
coords = df[["latitude", "longitude"]]
coords


In [ ]:
# Row selection by index
print(df.iloc[0])          # first row
print(df.iloc[1:4])       # rows 1–3

# Row selection by condition
high_stations = df[df["elevation_m"] >= 300]
high_stations


## 5. Sorting and filtering
Sorting and filtering are extremely common in catalog and metadata workflows.


In [ ]:
# Sort by elevation
df_sorted = df.sort_values("elevation_m", ascending=False)
df_sorted


In [ ]:
# Filter by start year
old_stations = df[df["start_year"] <= 1998]
old_stations


## 6. Basic summary statistics
Pandas provides many built-in summary operations.


In [ ]:
# Summary for numeric columns
df.describe()


In [ ]:
# Column-wise statistics
print("Mean elevation:", df["elevation_m"].mean())
print("Max elevation :", df["elevation_m"].max())
print("Min elevation :", df["elevation_m"].min())


## 7. Adding derived columns
It is common to compute new columns from existing ones.


In [ ]:
# Example: approximate distance from a reference point
lat0, lon0 = 16.72, -62.18

# crude distance in km (flat Earth approximation for teaching)
df["dist_km"] = 111.0 * np.sqrt((df["latitude"] - lat0)**2 + (df["longitude"] - lon0)**2)

df


## 8. Grouping and aggregation
Grouping is essential for summarizing by station, day, phase type, etc.


In [ ]:
# Create a tiny synthetic event table
events = pd.DataFrame({
    "station": ["MBGH", "MBGH", "MBBY", "MBLG", "MBGH", "MBBY"],
    "magnitude": [2.1, 3.4, 1.8, 2.9, 4.2, 3.0]
})

events


In [ ]:
# Mean magnitude by station
events.groupby("station")["magnitude"].mean()


### Joining tables (metadata + events)
Very common pattern: join catalogs with station metadata.


In [ ]:
merged = pd.merge(events, df, on="station", how="left")
merged


## 9. Plotting simple summaries from tables
Quick diagnostic plots are invaluable for QC.


In [ ]:
plt.figure()
plt.bar(df["station"], df["elevation_m"])
plt.ylabel("Elevation (m)")
plt.title("Station elevations")
plt.show()


In [ ]:
plt.figure()
plt.hist(events["magnitude"], bins=5)
plt.xlabel("Magnitude")
plt.ylabel("Count")
plt.title("Event magnitude distribution")
plt.show()


## 10. Common pitfalls and good habits
- Always print `df.shape` after reading a file
- Inspect `df.head()` before trusting the data
- Be explicit about column names
- Avoid silent type conversion errors
- Keep raw tables and derived tables separate


## Exercises
1. Load the CSV and select only stations above 250 m elevation.
2. Compute the mean elevation of those stations.
3. Add a column `is_high` that is True for elevation >= 300 m.
4. Group the `events` table by station and compute:
   - count of events
   - mean magnitude


In [ ]:
# 1) Filter stations above 250 m
high = None

# 2) Mean elevation of high stations
mean_high = None

# 3) Add is_high column
df2 = df.copy()
# TODO

# 4) Group and summarize events
summary = None

print("high:\n", high)
print("mean_high:", mean_high)
print("df2:\n", df2)
print("summary:\n", summary)


### Solutions (peek after trying)


In [ ]:
high = df[df["elevation_m"] > 250]
mean_high = high["elevation_m"].mean()

df2 = df.copy()
df2["is_high"] = df2["elevation_m"] >= 300

summary = events.groupby("station")["magnitude"].agg(["count", "mean"])

print("high:\n", high)
print("mean_high:", mean_high)
print("df2:\n", df2)
print("summary:\n", summary)
